In [1]:
import random
import copy
from mcts import mcts

# Setup

In [2]:
def next_piece():
    return random.choice(['R', 'G', 'B', 'Y']), random.choice(['R', 'G', 'B', 'Y'])

In [3]:
pieces = [next_piece() for _ in range(100)]
board = [[" " for _ in range(4)] for _ in range(9)]

In [4]:
INDIVIDUAL_LINK_POINTS = {
    0: 0,
    1: 40,
    2: 320,
    3: 640,
    4: 1280,
    5: 2560,
    6: 3840,
    7: 5120,
    8: 6400,
    9: 7680,
    10: 8960,
    11: 10240,
    12: 11520,
    13: 12800,
    14: 14080,
    15: 15360,
    16: 16640,
    17: 17920,
    18: 19200,
    19: 20480
}

In [5]:
# TODO: add color/grouping functionality/logic later
TOTAL_CHAIN_POINTS = {
    0: 0,
    1: 40,
    2: 360,
    3: 1000,
    4: 2280,
    5: 4840,
    6: 8680,
    7: 13800,
    8: 20200,
    9: 27880,
    10: 36840,
    11: 47080,
    12: 58600,
    13: 71400,
    14: 85480,
    15: 100840,
    16: 117480,
    17: 135400,
    18: 154600,
    19: 175080
}


# Puyo Env

In [24]:
class PuyoGameState:
    
    def __init__(self, board, pieces, score=0, turn=0):
        self.board = board
        self.board_height = len(self.board)
        self.board_columns = len(self.board[0])
        self.pieces = pieces  # List of (color1, color2) tuples
        self.score = score
        self.turn = turn  # Which piece we're on

    def __str__(self):
        return f"Turn: {self.turn}, Score: {self.score}\n" + "\n".join("".join(row) for row in self.board)

    def getPossibleActions(self):
        """ Required """
        """ Returns list of valid moves in ((column1, row1, color1), (column2, row2, color2)) format"""
        piece = self.pieces[self.turn]
        col1, col2 = piece
        col1, col2 = tuple(col1), tuple(col2)
        moves = []

        # For simplicity, assume 4 rotations: 0 = up, 1 = right, 2 = down, 3 = left
        for rotation in range(4):
            for column in range(self.board_columns):
                pos1, pos2, valid = self.placement_position(column, rotation, self.board)
                if valid:
                    moves.append((pos1 + col1, pos2 + col2))
        return moves
    
    def no_possible_moves_check(self):
        """ Checks if there are remaining moves for isTerminal() """
        # For simplicity, assume 4 rotations: 0 = up, 1 = right, 2 = down, 3 = left
        for rotation in range(4):
            for column in range(self.board_columns):
                _, _, valid = self.placement_position(column, rotation, self.board)
                if valid:
                    return False
        return True

    def placement_position(self, column, rotation, board):
        """Returns ((col1, row1), (col2, row2), valid_position)"""
        # 1: Check that we do not go out of the left/right bounds
        if column == 0:
            if rotation == 3:
                return ((0, 0), (0, 0), False)
        elif column == self.board_columns - 1:
            if rotation == 1:
                return ((0, 0), (0, 0), False)
            
        p1_height = self.get_column_height(column, board)
            
        # 2: Vertical placement
        # make sure that there is enough vertical space
        if rotation == 0:
            return ((column, p1_height-1), (column, p1_height-2), p1_height - 2 >= 0)
        elif rotation == 2:
            return ((column, p1_height-2), (column, p1_height-1), p1_height - 2 >= 0)
        
        # 3: Horizontal placement
        else:
            if rotation == 1:
                p2_height = self.get_column_height(column + 1, board)
                if (p1_height - 1 >= 0 and p2_height - 1 >= 0):
                    return ((column, p1_height-1), (column+1, p2_height-1), True)
                else:
                    return ((0, 0), (0, 0), False)
                
            elif rotation == 3:
                p2_height = self.get_column_height(column - 1, board)
                if (p1_height - 1 >= 0 and p2_height - 1 >= 0):
                    return ((column, p1_height-1), (column-1, p2_height-1), True)
                else:
                    return ((0, 0), (0, 0), False)
            else:
                raise ValueError(f"Invalid rotation value: {rotation}. Expected 0, 1, 2, or 3")

    def get_column_height(self, column, board):
        for row in range(self.board_height):
            if board[row][column] != " ":
                return row
        return self.board_height # empty column

    def takeAction(self, move):
        """ Required """
        if move is None:
            raise ValueError("Attempted to apply a None move.") # i dont think this will ever trigger. keep a lookout.
        
        new_board = copy.deepcopy(self.board)
        chain_length = 0
        # Drop the current piece into the board
        self.place_piece(new_board, move)
        # Find chains
        chained = self.find_chained_puyos(new_board)
        # Iterate until no more chains
        while chained:
            chain_length += 1
            self.remove_puyo(new_board, chained)
            self.apply_gravity(new_board)
            chained = self.find_chained_puyos(new_board)

        new_score = self.score + TOTAL_CHAIN_POINTS[chain_length]

        return PuyoGameState(new_board, self.pieces, new_score, turn=self.turn + 1)

    def isTerminal(self):
        """ Required """
        # Top-out check or no more pieces
        return self.board[0][1] != " " or self.turn >= len(self.pieces) or self.no_possible_moves_check()

    def getReward(self):
        """ Required """
        # Simple scoring mechanism for now
        return self.score

    def place_piece(self, board, piece):
        (col1, row1, color1), (col2, row2, color2) = piece
        board[row1][col1] = color1
        board[row2][col2] = color2

    def remove_puyo(self, board, chained_puyos):
        for group in chained_puyos:
            for row, col in group:
                board[row][col] = " "

    def apply_gravity(self, board):
        for col in range(self.board_columns):
            queue = []
            for row in reversed(range(self.board_height)):
                # Collect non-empty cells from bottom to top
                if board[row][col] != " ":
                    queue.append(board[row][col])
            # Fill up the column again from the stack
            for row in reversed(range(self.board_height)):
                board[row][col] = queue.pop(0) if queue else " "

    def find_chained_puyos(self, board):
        """ Uses standard DFS algo to find chained puyos """
        """ Returns a list of lists, each individual list contains the cells of the group of chained puyos"""
        visited = [[False for _ in range(self.board_columns)] for _ in range(self.board_height)]
        chained_puyos = []

        for row in range(self.board_height):
            for col in range(self.board_columns):
                if board[row][col] != " " and not visited[row][col]:
                    color = board[row][col]
                    group = []
                    stack = [(row, col)]

                    while stack:
                        crow, ccol = stack.pop()
                        if not (0 <= crow < self.board_height and 0 <= ccol < self.board_columns):
                            continue
                        if visited[crow][ccol] or board[crow][ccol] != color:
                            continue

                        visited[crow][ccol] = True
                        group.append((crow, ccol))

                        for drow, dcol in [(-1,0), (1,0), (0,-1), (0,1)]:
                            stack.append((crow + drow, ccol + dcol))

                    if len(group) >= 4:
                        chained_puyos.append(group)

        return chained_puyos


def print_board(board):
    emoji_map = {
        'R': '🟥',
        'G': '🟩',
        'B': '🟦',
        'Y': '🟨',
        'P': '🟪',
        ' ': '⬛'
    }

    for row in board:
        print('|' + ''.join(emoji_map.get(cell, '?!') for cell in row) + '|')
    print(" ")


# MCTS

In [30]:
pieces = [next_piece() for _ in range(25)]
board = [[" " for _ in range(4)] for _ in range(9)]

In [37]:
# Convert to expected format: [(c1, c2), ...] — already matches
initial_state = PuyoGameState(board, pieces)

# Initialize MCTS
searcher = mcts(timeLimit=50000)  # 1000 ms per move

current_state = initial_state
while not current_state.isTerminal():
    best_action = searcher.search(initialState=current_state)
    print(f"Best move for Turn {current_state.turn}: {best_action}")
    current_state = current_state.takeAction(best_action)
    print_board(current_state.board)
    print(f"Score so far: {current_state.score}\n")

print("Game Over!")
print(f"Final Score: {current_state.score}")


Best move for Turn 0: ((3, 8, 'B'), (3, 7, 'B'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
 
Score so far: 0

Best move for Turn 1: ((1, 8, 'B'), (2, 8, 'Y'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟦|
|⬛🟦🟨🟦|
 
Score so far: 0

Best move for Turn 2: ((2, 7, 'R'), (1, 7, 'R'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛🟥🟥🟦|
|⬛🟦🟨🟦|
 
Score so far: 0

Best move for Turn 3: ((0, 8, 'B'), (1, 6, 'G'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛🟩⬛⬛|
|⬛🟥🟥🟦|
|🟦🟦🟨🟦|
 
Score so far: 0

Best move for Turn 4: ((2, 6, 'R'), (2, 5, 'G'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛🟩⬛|
|⬛🟩🟥⬛|
|⬛🟥🟥🟦|
|🟦🟦🟨🟦|
 
Score so far: 0

Best move for Turn 5: ((1, 4, 'Y'), (1, 5, 'Y'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛🟨⬛⬛|
|⬛🟨🟩⬛|
|⬛🟩🟥⬛|
|⬛🟥🟥🟦|
|🟦🟦🟨🟦|
 
Score so far: 0

Best move for Turn 6: ((2, 4, 'G'), (2, 3, 'G'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛🟩⬛|
|⬛🟨🟩⬛|
|⬛🟨🟩⬛|
|⬛🟩🟥⬛|
|⬛🟥🟥🟦|
|🟦🟦🟨🟦|
 
Score so far: 0

Best move for Turn 7: ((3, 6, 'R'), (2, 2, 'Y'))
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|

In [ ]:
# Create an MCTS instance
searcher = mcts(timeLimit=1000)  # Time in ms to search for each move

# Start from your initial game state
initial_state = PuyoGameState(board, pieces)

# Use MCTS to search for the best move
best_move = searcher.search(
    initialState=initial_state
)

next_state = initial_state.takeAction(best_move)
print_board(next_state.board)

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|🟦🟨⬛⬛|
 


# Modular Tests

In [69]:
example_board = [
    [' ', ' ', ' ', 'Y'],
    [' ', ' ', 'G', 'Y'],
    [' ', ' ', 'B', 'G'],
    [' ', ' ', 'G', 'B'],
    [' ', ' ', 'B', 'B'],
    ['R', 'Y', 'R', 'G'],
    ['R', 'R', 'Y', 'R'],
    ['Y', 'Y', 'R', 'G'],
    ['G', 'G', 'R', 'G'],
]

print_board(example_board)
move = ((0, 4, 'R'), (0, 3, 'B'))

state = PuyoGameState(example_board, pieces)
new_state = state.do_move(move)

print_board(new_state.board)
print(new_state.score)

|⬛⬛⬛🟨|
|⬛⬛🟩🟨|
|⬛⬛🟦🟩|
|⬛⬛🟩🟦|
|⬛⬛🟦🟦|
|🟥🟨🟥🟩|
|🟥🟥🟨🟥|
|🟨🟨🟥🟩|
|🟩🟩🟥🟩|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟨|
|🟦⬛⬛🟨|
 
8680


In [72]:
example_board = [
    [' ', ' ', ' ', 'Y'],
    [' ', ' ', 'G', 'Y'],
    [' ', ' ', 'B', 'G'],
    [' ', ' ', 'G', 'B'],
    [' ', ' ', 'B', 'B'],
    ['R', 'Y', 'R', 'G'],
    ['R', 'R', 'Y', 'R'],
    ['Y', 'Y', 'R', 'G'],
    ['G', 'G', 'R', 'G'],
]

print_board(example_board)
#piece = ((0, 4, 'R'), (0, 3, 'B'))
piece = [('a', 'b')]

state = PuyoGameState(example_board, piece)

state.get_moves()


|⬛⬛⬛🟨|
|⬛⬛🟩🟨|
|⬛⬛🟦🟩|
|⬛⬛🟩🟦|
|⬛⬛🟦🟦|
|🟥🟨🟥🟩|
|🟥🟥🟨🟥|
|🟨🟨🟥🟩|
|🟩🟩🟥🟩|
 


[((0, 4, 'a'), (0, 3, 'b')),
 ((1, 4, 'a'), (1, 3, 'b')),
 ((0, 4, 'a'), (1, 4, 'b')),
 ((1, 4, 'a'), (2, 0, 'b')),
 ((0, 3, 'a'), (0, 4, 'b')),
 ((1, 3, 'a'), (1, 4, 'b')),
 ((1, 4, 'a'), (0, 4, 'b')),
 ((2, 0, 'a'), (1, 4, 'b'))]

In [ ]:
example_board = [
    [' ', ' ', ' ', ' '],
    [' ', ' ', ' ', ' '],
    [' ', 'G', ' ', 'R'],
    [' ', 'G', ' ', 'R'],
    ['R', ' ', ' ', 'B'],
    ['R', ' ', ' ', 'B'],
    ['G', ' ', ' ', 'B'],
    ['G', 'Y', 'B', 'Y'],
    ['R', 'R', 'Y', 'Y'],
]

pieces = []  # No pieces needed for this test

# Instantiate game state
state = PuyoGameState(example_board, pieces)
print_board(state.board)
# Testing of functions
# find chains, should be empty
chained = state.find_chained_puyos(state.board)
print(f"Chained puyos: {chained}\n")

# Step 1: Apply gravity
state.apply_gravity(state.board)
print_board(state.board)

# Step 2: Find chains
chained = state.find_chained_puyos(state.board) # empty output
print(f"Chained puyos: {chained}")

# Step 3: Pop puyos
state.remove_puyo(state.board, chained)
print_board(state.board)

print("=================================")
# Loop until no chains
while chained:
    state.apply_gravity(state.board)
    print_board(state.board)
    chained = state.find_chained_puyos(state.board)
    print(f"Chained puyos: {chained}\n")
    state.remove_puyo(state.board, chained)
    print_board(state.board)


|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛🟩⬛🟥|
|⬛🟩⬛🟥|
|🟥⬛⬛🟦|
|🟥⬛⬛🟦|
|🟩⬛⬛🟦|
|🟩🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: []

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|🟥⬛⬛🟦|
|🟥🟩⬛🟦|
|🟩🟩⬛🟦|
|🟩🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: [[(5, 1), (6, 1), (6, 0), (7, 0)]]
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|🟥⬛⬛🟦|
|🟥⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛🟨🟦🟨|
|🟥🟥🟨🟨|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|🟥⬛⬛🟦|
|🟥🟨🟦🟨|
|🟥🟥🟨🟨|
 
Chained puyos: [[(6, 0), (7, 0), (8, 0), (8, 1)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛🟨🟦🟨|
|⬛⬛🟨🟨|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦🟨|
|⬛🟨🟨🟨|
 
Chained puyos: [[(7, 3), (8, 3), (8, 2), (8, 1)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦⬛|
|⬛⬛⬛⬛|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛🟦|
|⬛⬛⬛🟦|
|⬛⬛🟦🟦|
 
Chained puyos: [[(6, 3), (7, 3), (8, 3), (8, 2)]]

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
 
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
 
Chained puyos: []

|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛⬛|
|⬛⬛⬛🟥|
|⬛⬛⬛🟥|
 


To work with the mcts package, your game state class must implement:

Method	Description
get_moves()	Returns list of legal moves from this state
do_move(move)	Returns the new state after applying a move
is_terminal()	Returns True if game is over (e.g. top-out)
get_result()	Returns a numeric value or score for this state